In [1]:
from jax.config import config
config.update("jax_enable_x64", True)

import jax.numpy as jnp
import numpy as onp
import matplotlib.pyplot as plt
%matplotlib inline

%load_ext autoreload
%autoreload 2

In [2]:
from gen_elo.margin_functions import margin_functions

In [3]:
from tdata.datasets.oncourt_dataset import OnCourtDataset

In [4]:
dataset = OnCourtDataset()

/anaconda2/envs/jax/lib/python3.7/site-packages/pandas/core/indexing.py:376: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/anaconda2/envs/jax/lib/python3.7/site-packages/pandas/core/indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [5]:
df = dataset.get_stats_df()

In [6]:
to_use = df[df['year'] >= 2010]
to_use = to_use.dropna(subset=['W1S_1'])

spw_1 = (to_use['W1S_1'] + to_use['W2S_1']) / (to_use['W1SOF_1'] + to_use['W2SOF_1'])
spw_2 = (to_use['W1S_2'] + to_use['W2S_2']) / (to_use['W1SOF_2'] + to_use['W2SOF_2'])

margins = (spw_1 - spw_2).values

In [7]:
winners = to_use['winner'].values
losers = to_use['loser'].values
surface = to_use['surface'].values

In [8]:
from sklearn.preprocessing import LabelEncoder

In [9]:
surf_encoder = LabelEncoder()

In [10]:
surface_enc = surf_encoder.fit_transform(surface)

In [11]:
n_matches = winners.shape[0]

a_winner = onp.zeros((n_matches, len(surf_encoder.classes_)))
a_winner[onp.arange(n_matches), surface_enc] = 1
a_loser = a_winner.copy()

a_full = onp.concatenate([a_winner, -a_loser], axis=1)

In [12]:
a_full.shape

(26039, 8)

In [13]:
import numpy as onp

encoder = LabelEncoder()
encoder.fit(onp.concatenate([winners, losers]))

winners_array, losers_array = map(encoder.transform, [winners, losers])

In [14]:
from gen_elo.general import optimise_elo, EloParams

In [15]:
theta = {'factor': 0.1, 'obs_sd': 0.1, 'offset': 0.1}
theta = {x: jnp.array(y) for x, y in theta.items()}

start_params = EloParams(
    theta=theta,
    cov_mat=jnp.eye(len(surf_encoder.classes_))
)

/anaconda2/envs/jax/lib/python3.7/site-packages/jax/lib/xla_bridge.py:114: UserWarning: No GPU/TPU found, falling back to CPU.
  warnings.warn('No GPU/TPU found, falling back to CPU.')


In [16]:
y_full = margins.reshape(-1, 1)

In [17]:
final_params, success = optimise_elo(start_params, margin_functions, winners_array,
                      losers_array, a_full, y_full, n_players=len(encoder.classes_))

{'factor': Traced<ShapedArray(float64[])>with<JVPTrace(level=1/1)>, 'obs_sd': Traced<ShapedArray(float64[])>with<JVPTrace(level=1/1)>, 'offset': Traced<ShapedArray(float64[])>with<JVPTrace(level=1/1)>}
Traced<ShapedArray(float64[4,4])>with<JVPTrace(level=1/1)>
{'factor': 0.010000000000000002, 'obs_sd': 0.010000000000000002, 'offset': array(0.1)}
[[1.000001 0.       0.       0.      ]
 [0.       1.000001 0.       0.      ]
 [0.       0.       1.000001 0.      ]
 [0.       0.       0.       1.000001]]
{'factor': 0.6564177739884723, 'obs_sd': 0.668912256526237, 'offset': array(0.09745148)}
[[1.02216464e+00 3.19189324e-05 2.83966290e-04 1.78532076e-04]
 [3.19189324e-05 1.00757878e+00 1.74245358e-04 8.73272521e-05]
 [2.83966290e-04 1.74245358e-04 1.02883542e+00 2.96455354e-04]
 [1.78532076e-04 8.73272521e-05 2.96455354e-04 1.01202081e+00]]
{'factor': 695.4671119268756, 'obs_sd': 805.2490658318138, 'offset': array(-0.00935368)}
[[126.30085424  -0.81504343  -2.10521777  -1.47350941]
 [ -0.815

{'factor': 0.9630282644591573, 'obs_sd': 0.5041690752891781, 'offset': array(0.03694274)}
[[ 0.02859491  0.0043972   0.01877873 -0.02531034]
 [ 0.0043972   0.00432442  0.02460587 -0.00686747]
 [ 0.01877873  0.02460587  0.32271342  0.17982836]
 [-0.02531034 -0.00686747  0.17982836  0.28074246]]
{'factor': 0.9577132649807826, 'obs_sd': 0.5084995805378214, 'offset': array(0.02403106)}
[[ 0.01226746  0.00187124 -0.00254299  0.00199839]
 [ 0.00187124  0.00405644  0.00312954  0.00167638]
 [-0.00254299  0.00312954  0.24481291  0.0196592 ]
 [ 0.00199839  0.00167638  0.0196592   0.00276809]]
{'factor': 0.9584255382912019, 'obs_sd': 0.5078431123993659, 'offset': array(0.02475291)}
[[ 0.01251168  0.00188597 -0.00272255  0.00196773]
 [ 0.00188597  0.00402076  0.00299838  0.001633  ]
 [-0.00272255  0.00299838  0.24027829  0.01978706]
 [ 0.00196773  0.001633    0.01978706  0.00283418]]
{'factor': 0.9612772792183467, 'obs_sd': 0.5052214800537406, 'offset': array(0.02764031)}
[[ 0.01351263  0.00194412

{'factor': 1.9356592257429917, 'obs_sd': 0.07176895284326427, 'offset': array(0.09333156)}
[[ 7.10002342e-05  8.07132133e-05  7.26860303e-05  2.17565452e-04]
 [ 8.07132133e-05  3.21411994e-04 -3.65383791e-04  4.98660767e-04]
 [ 7.26860303e-05 -3.65383791e-04  1.26322934e-03 -2.89840416e-04]
 [ 2.17565452e-04  4.98660767e-04 -2.89840416e-04  9.59665391e-04]]
{'factor': 1.8931096586040626, 'obs_sd': 0.08043614882998891, 'offset': array(0.09910122)}
[[ 1.27433311e-05  1.60514149e-05  3.20381187e-05  3.31620520e-05]
 [ 1.60514149e-05  1.06846015e-04 -2.27697698e-04  3.07999895e-04]
 [ 3.20381187e-05 -2.27697698e-04  1.17450413e-03 -6.72744162e-04]
 [ 3.31620520e-05  3.07999895e-04 -6.72744162e-04  9.77669622e-04]]
{'factor': 1.8599775218207333, 'obs_sd': 0.08751379695138718, 'offset': array(0.10372308)}
[[ 1.77870119e-05  3.04005817e-05  1.96694409e-05  5.99469815e-05]
 [ 3.04005817e-05  5.60858489e-05  3.10302194e-05  1.11711025e-04]
 [ 1.96694409e-05  3.10302194e-05  7.94444652e-04 -3.51

{'factor': 1.8596891689688175, 'obs_sd': 0.08765984900326342, 'offset': array(0.10225079)}
[[7.26536251e-05 2.83954721e-05 4.88263800e-05 3.20822040e-05]
 [2.83954721e-05 7.38262893e-05 4.93384361e-05 4.77633456e-05]
 [4.88263800e-05 4.93384361e-05 6.20585116e-05 5.33675256e-05]
 [3.20822040e-05 4.77633456e-05 5.33675256e-05 5.11587303e-05]]
{'factor': 1.8594985775542914, 'obs_sd': 0.08766531531037618, 'offset': array(0.10224872)}
[[7.27596615e-05 2.84454334e-05 4.89194566e-05 3.20479043e-05]
 [2.84454334e-05 7.40982865e-05 4.93632554e-05 4.78026262e-05]
 [4.89194566e-05 4.93632554e-05 6.22244484e-05 5.34550358e-05]
 [3.20479043e-05 4.78026262e-05 5.34550358e-05 5.12378609e-05]]
{'factor': 1.8592107105656754, 'obs_sd': 0.08766759948504464, 'offset': array(0.10224824)}
[[7.28971011e-05 2.84897880e-05 4.89664318e-05 3.20304664e-05]
 [2.84897880e-05 7.43928558e-05 4.94113135e-05 4.77897001e-05]
 [4.89664318e-05 4.94113135e-05 6.23647889e-05 5.35046706e-05]
 [3.20304664e-05 4.77897001e-05 

{'factor': 0.12256858005329209, 'obs_sd': 0.0873933792570606, 'offset': array(0.10305501)}
[[0.00267036 0.00145518 0.00252894 0.0023353 ]
 [0.00145518 0.00197653 0.00251692 0.00301404]
 [0.00252894 0.00251692 0.00363346 0.00349795]
 [0.0023353  0.00301404 0.00349795 0.00705138]]
{'factor': 0.08343708266335811, 'obs_sd': 0.08800076041124931, 'offset': array(0.10167744)}
[[0.00401302 0.00248971 0.00421266 0.00412065]
 [0.00248971 0.00284208 0.00416369 0.00509296]
 [0.00421266 0.00416369 0.00700498 0.00627551]
 [0.00412065 0.00509296 0.00627551 0.01324078]]
{'factor': 0.06623970121398208, 'obs_sd': 0.08895355214908401, 'offset': array(0.09917422)}
[[0.00643506 0.00453781 0.00749706 0.00771071]
 [0.00453781 0.00456183 0.00742232 0.00916033]
 [0.00749706 0.00742232 0.01475416 0.01224333]
 [0.00771071 0.00916033 0.01224333 0.0264159 ]]
{'factor': 0.034156287954440756, 'obs_sd': 0.09048428376692678, 'offset': array(0.09711093)}
[[0.01411743 0.01139637 0.0184062  0.01983601]
 [0.01139637 0.010

{'factor': 0.024836274405269265, 'obs_sd': 0.0883843738800491, 'offset': array(0.10255711)}
[[0.15753507 0.16911177 0.1551323  0.19933325]
 [0.16911177 0.18154711 0.16677909 0.2150311 ]
 [0.1551323  0.16677909 0.22202958 0.1918924 ]
 [0.19933325 0.2150311  0.1918924  0.48412254]]
{'factor': 0.025010557886442644, 'obs_sd': 0.08844439273909185, 'offset': array(0.10224775)}
[[0.15950545 0.16854682 0.15667724 0.20083813]
 [0.16854682 0.17812703 0.16607529 0.21438748]
 [0.15667724 0.16607529 0.22641102 0.19537763]
 [0.20083813 0.21438748 0.19537763 0.48407959]]
{'factor': 0.025276295383611674, 'obs_sd': 0.08853389813359545, 'offset': array(0.10178097)}
[[0.1632278  0.16722059 0.15912772 0.20318465]
 [0.16722059 0.17140807 0.16408029 0.21248586]
 [0.15912772 0.16408029 0.23320922 0.20110358]
 [0.20318465 0.21248586 0.20110358 0.48201758]]
{'factor': 0.02562699483116954, 'obs_sd': 0.088651957579977, 'offset': array(0.10115884)}
[[0.17015196 0.16441513 0.16277059 0.20662055]
 [0.16441513 0.159

{'factor': 0.02598175556502709, 'obs_sd': 0.0883346849396317, 'offset': array(0.10080258)}
[[0.2574581  0.14891033 0.16491685 0.16997285]
 [0.14891033 0.20086182 0.12117764 0.19833084]
 [0.16491685 0.12117764 0.30189157 0.22859278]
 [0.16997285 0.19833084 0.22859278 0.32216265]]
{'factor': 0.02597245709347136, 'obs_sd': 0.0882469988796858, 'offset': array(0.10080399)}
[[0.24510828 0.14787047 0.15789741 0.16156752]
 [0.14787047 0.26005841 0.12898325 0.19654059]
 [0.15789741 0.12898325 0.31988458 0.21843104]
 [0.16156752 0.19654059 0.21843104 0.26159223]]
{'factor': 0.025238124199163374, 'obs_sd': 0.08804272752449045, 'offset': array(0.10150375)}
[[0.24336172 0.1540962  0.15454007 0.15832032]
 [0.1540962  0.2963584  0.13122507 0.19345855]
 [0.15454007 0.13122507 0.31679758 0.2039932 ]
 [0.15832032 0.19345855 0.2039932  0.23308897]]
{'factor': 0.024622771113931994, 'obs_sd': 0.08781255132877776, 'offset': array(0.10232229)}
[[0.24506137 0.15947117 0.15056778 0.15425957]
 [0.15947117 0.308

{'factor': 0.02469446271311046, 'obs_sd': 0.0878173655425014, 'offset': array(0.10217697)}
[[0.26121273 0.16769211 0.13236582 0.1425408 ]
 [0.16769211 0.362476   0.17147326 0.21631908]
 [0.13236582 0.17147326 0.25186774 0.18249574]
 [0.1425408  0.21631908 0.18249574 0.21978256]]
{'factor': 0.02465707143190135, 'obs_sd': 0.08780285216137648, 'offset': array(0.10221618)}
[[0.25999144 0.16473469 0.12660297 0.14429628]
 [0.16473469 0.37034644 0.18877766 0.23196811]
 [0.12660297 0.18877766 0.24577119 0.19411785]
 [0.14429628 0.23196811 0.19411785 0.23983923]]
{'factor': 0.02463842714702508, 'obs_sd': 0.08778558829645411, 'offset': array(0.10226049)}
[[0.25283665 0.16055462 0.12385279 0.1501823 ]
 [0.16055462 0.37442325 0.19364374 0.23990065]
 [0.12385279 0.19364374 0.24107565 0.2025123 ]
 [0.1501823  0.23990065 0.2025123  0.25808776]]
{'factor': 0.024655376589779735, 'obs_sd': 0.08777702761302322, 'offset': array(0.10227381)}
[[0.2483157  0.15855572 0.12395357 0.15404486]
 [0.15855572 0.375

{'factor': 0.024493862263710225, 'obs_sd': 0.08778766166399221, 'offset': array(0.10221467)}
[[0.2807243  0.11057671 0.17873967 0.17380058]
 [0.11057671 0.34262603 0.22927461 0.21601544]
 [0.17873967 0.22927461 0.22672372 0.19487659]
 [0.17380058 0.21601544 0.19487659 0.2451239 ]]
{'factor': 0.024483815736036803, 'obs_sd': 0.08778858817421038, 'offset': array(0.10221879)}
[[0.27710987 0.11385326 0.18090725 0.17044731]
 [0.11385326 0.34981092 0.23268732 0.21842514]
 [0.18090725 0.23268732 0.22773046 0.19665002]
 [0.17044731 0.21842514 0.19665002 0.24340102]]
{'factor': 0.024503725451780242, 'obs_sd': 0.0877870714688268, 'offset': array(0.10220587)}
[[0.27279184 0.11691387 0.18063816 0.16587526]
 [0.11691387 0.35839812 0.23424343 0.22093896]
 [0.18063816 0.23424343 0.22801938 0.19902418]
 [0.16587526 0.22093896 0.19902418 0.243388  ]]
{'factor': 0.024568419802188577, 'obs_sd': 0.08778368659447741, 'offset': array(0.10217031)}
[[0.26871799 0.11871762 0.17743195 0.16116378]
 [0.11871762 0.

In [18]:
success

True

In [19]:
from gen_elo.general import calculate_ratings_scan

final_ratings, total_lik = calculate_ratings_scan(
    winners_array, losers_array, a_full, y_full, margin_functions,
    final_params, jnp.zeros((len(encoder.classes_), a_full.shape[1] // 2)))

In [20]:
import pandas as pd

final_ratings = pd.DataFrame(final_ratings, index=encoder.classes_, columns=surf_encoder.classes_).drop(
    ['Robin Soderling','Andy Roddick','Mardy Fish', 'David Nalbandian'], errors='ignore')

In [21]:
from tpr.models.utils import to_elo_scale

In [22]:
final_ratings_elo = to_elo_scale(final_ratings)

In [23]:
final_ratings_elo.sort_values('indoor_hard', ascending=False)['indoor_hard'].round(0).astype(int).iloc[:20]

Novak Djokovic           2145
Rafael Nadal             2145
Roger Federer            2140
Daniil Medvedev          2053
Juan Martin Del Potro    1976
Dominic Thiem            1906
Milos Raonic             1873
Stefanos Tsitsipas       1864
Alexander Zverev         1860
Andrey Rublev            1853
Stan Wawrinka            1845
Denis Shapovalov         1838
Grigor Dimitrov          1837
Kei Nishikori            1832
Matteo Berrettini        1828
Jo-Wilfried Tsonga       1828
David Goffin             1809
Roberto Bautista Agut    1805
Jan-Lennard Struff       1804
Kevin Anderson           1790
Name: indoor_hard, dtype: int64

In [24]:
# TODO: Also make some utils for betting, like expected value and so on.

In [25]:
from ml_tools.normals import covar_to_corr

pd.DataFrame(covar_to_corr(final_params.cov_mat), index=surf_encoder.classes_, columns=surf_encoder.classes_).round(2)

,clay,grass,hard,indoor_hard
clay,1.00,0.41,0.71,0.64
grass,0.41,1.00,0.82,0.72
hard,0.71,0.82,1.00,0.86
indoor_hard,0.64,0.72,0.86,1.00


In [26]:
surface = 'indoor_hard'

p1 = 'Stefanos Tsitsipas'
p2 = 'Alexander Zverev'

p1_mean, p2_mean = final_ratings.loc[p1].values, final_ratings.loc[p2].values

In [27]:
a_surf = jnp.array([0, 0, 0, 1])
cur_a = jnp.concatenate([a_surf, -a_surf])

In [28]:
from ml_tools.jax import weighted_sum, logistic_normal_integral_approx

latent_mean, latent_var = weighted_sum(jnp.concatenate([p1_mean, p2_mean]), 
                                       jnp.kron(jnp.eye(2), final_params.cov_mat), cur_a)

In [29]:
p1_win_prob = logistic_normal_integral_approx(latent_mean, latent_var)

p1_win_prob

DeviceArray(0.50583331)

In [30]:
from tpr.models.utils import odds_prob_equal_allocation

In [31]:
1 / (1 - p1_win_prob)

DeviceArray(2.02360868)

In [32]:
def calculate_good_odds(p_win, min_gain=0.1):
    
    return (min_gain + 1) / p_win

In [33]:
calculate_good_odds(p1_win_prob)

DeviceArray(2.17462942)